In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [3]:
train_df = pd.read_csv('features.csv', index_col='match_id')
test_df = pd.read_csv('features_test.csv', index_col='match_id')
train_df.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time,duration,radiant_win,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,4,2,2,-52,2874,1,1796,0,51,0
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,4,3,1,-5,2463,1,1974,0,63,1
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,4,3,1,13,2130,0,0,1830,0,63
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,4,2,0,27,1459,0,1920,2047,50,63
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,3,3,0,-16,2449,0,4,1974,3,63


# Подход 1: градиентный бустинг "в лоб"

Исключаем целевую переменную 'radiant_win' и признаки итогов матча

In [4]:
train_x, train_y = train_df.drop(['radiant_win', 'duration', 'tower_status_radiant' , 'tower_status_dire', 
                           'barracks_status_radiant', 'barracks_status_dire'], axis=1), train_df['radiant_win']

Выведем колонки с пропущенными признаками

In [5]:
count_values = train_x.count()
count_values[count_values < train_x.shape[0]]

first_blood_time               77677
first_blood_team               77677
first_blood_player1            77677
first_blood_player2            53243
radiant_bottle_time            81539
radiant_courier_time           96538
radiant_flying_courier_time    69751
radiant_first_ward_time        95394
dire_bottle_time               81087
dire_courier_time              96554
dire_flying_courier_time       71132
dire_first_ward_time           95404
dtype: int64

Пропущены данные по признакам 'first_blood', и признаки покупок для команд 'radiant' и 'dire'. Событие 'first_blood' означает, что на момент записи признака не произошли события "первая кровь"(никто ни кого не ранил). А остальные говорит об отсутствии покупки предметов ('bottle', 'carrier', 'flying courier') командами 'Radiant' и 'Dire' соответственно. Отсутствие признака 'ward_time' говорит о том, что команда не установила наблюдателя на поле.

Заменем пропуски 0

In [6]:
train_x = train_x.fillna(value=0)

In [7]:
RANDOM_STATE = 1
cv = KFold(train_x.shape[0], n_folds=5, shuffle=True, random_state=RANDOM_STATE)
for n_trees in (10, 20, 30):
    print 'Trees: {:}'.format(n_trees)
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=n_trees, random_state=RANDOM_STATE)
    cv_res = cross_val_score(clf, train_x, train_y, cv=cv, scoring='roc_auc', n_jobs=4)
    mean_acc = cv_res.mean()
    print 'Time elapsed:', datetime.datetime.now() - start_time
    print 'Acc.: {:.5f}'.format(mean_acc)        

Trees: 10
Time elapsed: 0:00:56.140000
Acc.: 0.66483
Trees: 20
Time elapsed: 0:01:41.765000
Acc.: 0.68211
Trees: 30
Time elapsed: 0:02:24.372000
Acc.: 0.68969


Таким образом для тренировки классификатора на 30 деревьях понадобилось 2 минуты 24 секунд. Качество модели при этом составило 0.68969. Дальнейшее увеличение количества деревьев не дает существенного прироста точности. Для ускорения обучения можно настроить параметры классификатора, как-то максимальную глубину дерева 'max_depth', количество используемых признаков 'max_features' или количество данных в узле дерева (используя параметры 'min_samples_split' или 'min_samples_leaf').

# Подход 2: логистическая регрессия

In [8]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
RANDOM_STATE = 1
cv = KFold(train_x.shape[0], n_folds=5, shuffle=True, random_state=RANDOM_STATE)
clf = LogisticRegression(random_state=RANDOM_STATE)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=4)
gs.fit(train_x, train_y)
max_score, params = 0, None
for a in gs.grid_scores_:
    if a.mean_validation_score > max_score:
        max_score = a.mean_validation_score
        params = a.parameters
start_time = datetime.datetime.now()
clf = LogisticRegression(random_state=RANDOM_STATE, C=params['C'])
cv_res = cross_val_score(clf, train_x, train_y, cv=cv, scoring='roc_auc', n_jobs=4)
mean_acc = cv_res.mean()
print 'Time elapsed:', datetime.datetime.now() - start_time
print 'Acc.: {:.5f}'.format(mean_acc)       

Time elapsed: 0:00:06.067000
Acc.: 0.51347


На всех данных классификатор показал точность 0.51347, при этом обучение заняло 3 секунды. Классификатор обучается быстрее, но и качество модели явно меньше. На качество могли повлиять разные масштабы признаков. Уберем категориальные признаки и применим к признакам StandardScaler.

In [9]:
train_x_new = train_x.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(train_x_new)

In [10]:
cv = KFold(X_train_scaled.shape[0], n_folds=5, shuffle=True, random_state=RANDOM_STATE)
clf = LogisticRegression(random_state=RANDOM_STATE)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=4)
gs.fit(X_train_scaled, train_y)
max_score, params = 0, None
for a in gs.grid_scores_:
    if a.mean_validation_score > max_score:
        max_score = a.mean_validation_score
        params = a.parameters
start_time = datetime.datetime.now()
clf = LogisticRegression(random_state=RANDOM_STATE, C=params['C'])
cv_res = cross_val_score(clf, X_train_scaled, train_y, cv=cv, scoring='roc_auc', n_jobs=4)
mean_acc = cv_res.mean()
print 'Time elapsed:', datetime.datetime.now() - start_time
print 'Acc.: {:.5f}'.format(mean_acc)       

Time elapsed: 0:00:14.495000
Acc.: 0.71641


Точность классификатора повысилась до 0.71641. Время необходимое для обучение также выросло до 14 секунд. Категориальные признаки учитывались как числовые и оказывали значительное влияние на результат модели.

In [11]:
heroes = train_x[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].stack()
print heroes.value_counts().shape[0]
print heroes.max()

108
112


В игре 112 различных идентификаторов героев. В выборке есть 108

In [ ]:
X_pick = np.zeros((train_x.shape[0], 112))

for i, match_id in enumerate(train_x.index):
    for p in xrange(5):
        X_pick[i, train_x.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, train_x.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [13]:
from scipy.sparse import coo_matrix, hstack
train_x_new = hstack([coo_matrix(X_train_scaled), coo_matrix(X_pick)]).toarray()

In [ ]:
cv = KFold(train_x_new.shape[0], n_folds=5, shuffle=True, random_state=RANDOM_STATE)
clf = LogisticRegression(random_state=RANDOM_STATE)
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, n_jobs=4)
gs.fit(train_x_new, train_y)
max_score, params = 0, None
for a in gs.grid_scores_:
    if a.mean_validation_score > max_score:
        max_score = a.mean_validation_score
        params = a.parameters
start_time = datetime.datetime.now()
clf = LogisticRegression(random_state=RANDOM_STATE, C=params['C'])
cv_res = cross_val_score(clf, train_x_new, train_y, cv=cv, scoring='roc_auc', n_jobs=4)
mean_acc = cv_res.mean()
print 'Time elapsed:', datetime.datetime.now() - start_time
print 'Acc.: {:.5f}'.format(mean_acc)  

Time elapsed: 0:01:10.422000
Acc.: 0.75100


Теперь качество модели составляет 0.75187. Добавление мешка слов по героям дает прирост качества модели в сравнении с прошлым вариантом. Т.е. информация о героях влияет на результат анализа и мы не можем ее исключать. Логистическая регрессия дает более точные результаты. Так что применем модель к тестовой выборке.

In [17]:
clf = LogisticRegression(random_state=RANDOM_STATE, C=params['C'])
clf.fit(train_x_new, train_y)
test_x = test_df.fillna(value=0)
test_x_new = test_x.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 
                        'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis=1)
test_x_scaled = scaler.transform(test_x_new)
X_pick = np.zeros((test_x.shape[0], 112))

for i, match_id in enumerate(test_x.index):
    for p in xrange(5):
        X_pick[i, test_x.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, test_x.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
test_x_new = hstack([coo_matrix(test_x_scaled), coo_matrix(X_pick)]).toarray()

pred = clf.predict_proba(test_x_new)[:, 1]
print 'Min.: {:.5f}'.format(np.min(pred)) 
print 'Max.: {:.5f}'.format(np.max(pred)) 

Min.: 0.00843
Max.: 0.99637


Минимальное значение 0.00843. Максимальное 0.99637. 